In [1]:
import gzip, json
import random
import re
import pronouncing

In [2]:
all_lines =[]
for line in gzip.open("gutenberg-poetry-v001.ndjson.gz"):
    all_lines.append(json.loads(line.strip()))

In [3]:
#The following cell builds the data structure proposed above: a dictionary that maps rhyming parts 
#to a dictionary that maps words with that rhyming part to the lines of poetry that they're found at the end of.
from collections import defaultdict
by_rhyming_part = defaultdict(lambda: defaultdict(list))
for line in all_lines:
    text = line['s']
    if not(32 < len(text) < 48): # only use lines of uniform lengths
        continue
    match = re.search(r'(\b\w+\b)\W*$', text)
    if match:
        last_word = match.group()
        pronunciations = pronouncing.phones_for_word(last_word)
        if len(pronunciations) > 0:
               rhyming_part = pronouncing.rhyming_part(pronunciations[0])
               # group by rhyming phones (for rhymes) and words (to avoid duplicate words)
               by_rhyming_part[rhyming_part][last_word.lower()].append(text)
               
               

In [298]:
source_word = 'knelling'
print(source_word.isalpha())
phones = pronouncing.phones_for_word(source_word)#[0] #
print(phones)
rhyming_part = pronouncing.rhyming_part(phones)

True
[]


AttributeError: 'list' object has no attribute 'split'

In [296]:
print(len(by_rhyming_part[rhyming_part]))
by_rhyming_part[rhyming_part].keys()


52


dict_keys(['blaze', 'praise', 'raise', 'rase', 'ways', 'gaze', 'days', 'graze', 'amaze', 'stays', 'surveys', 'obeys', 'delays', "days'", 'phrase', 'strays', 'weighs', 'lays', 'betrays', 'displays', 'plays', 'maze', 'ablaze', 'haze', 'bays', 'rays', 'pays', 'claes', 'jays', 'bayes', 'craze', 'sways', 'prays', 'maize', 'glaze', 'essays', 'chaise', 'portrays', 'mays', 'conveys', 'decays', 'sprays', 'phase', 'repays', 'dismays', 'res', 'raze', 'appraise', 'grays', 'chalets', 'preys', 'baize'])

In [194]:
" ".isalpha()

False

In [310]:
line1, line2, line3, line4, line5 = limirick_machine()
print(line1)
print(line2)
print(line3)
print(line4)
print(line5)

UnboundLocalError: local variable 'line2' referenced before assignment

In [400]:
def limirick_machine():
    
    def initial_line_finder(list_of_existing):
        rand_choice = random.choice(all_lines)['s']
        if rand_choice not in list_of_existing:
            line = rand_choice
        while line.split(' ')[-1].isalpha() == False:
            rand_choice = random.choice(all_lines)['s']
            if rand_choice not in list_of_existing:
                line = rand_choice
        
        word_to_rhyme = line.split(' ')[-1]
        phones = pronouncing.phones_for_word(word_to_rhyme)
        phones = phones[0]
        rhyming_phones = pronouncing.rhyming_part(phones)
        return line, rhyming_phones
    
    
    def secondary_line_finder(list_of_existing, phones_to_rhyme):   
        key = random.choice(list(by_rhyming_part[phones_to_rhyme].keys()))    
        rand_choice = random.choice(list(by_rhyming_part[phones_to_rhyme][key]))       
        while rand_choice in list_of_existing:
            key = random.choice(list(by_rhyming_part[phones_to_rhyme].keys()))    
            rand_choice = random.choice(list(by_rhyming_part[phones_to_rhyme][key]))       
        return rand_choice
            

    list_of_lines = []
    line1, line1_phones = initial_line_finder(list_of_lines)
    list_of_lines.append(line1)
    
    ##########
    #Setting up line2, and the function to accomodate initiating and follow up.
    ###
    
    line2 = secondary_line_finder(list_of_lines, line1_phones)
    list_of_lines.append(line2)
    
    line3, line3_phones = initial_line_finder(list_of_lines)
    list_of_lines.append(line3)
    
    line4 = secondary_line_finder(list_of_lines, line3_phones)
    list_of_lines.append(line4)
    
    line5 = secondary_line_finder(list_of_lines, line1_phones)

    return line1, line2, line3, line4, line5

    
    

In [401]:
from textblob import TextBlob

In [415]:
line1, line2, line3, line4, line5 = limirick_machine()
print(line1)
print(line2)
print(line3)
print(line4)
print(line5)

And I am strong to drink the cup that drains
Urania, my celestial guest! who deigns
To warm the dead, and vainly light
Embraced is, and fully gan despite
By ASTRACAN over the Snowie Plaines


In [386]:
import pandas as pd
temp_limirick_dict= {'line1':[],'line2':[],'line3':[],'line4':[],'line5':[],}
for i in range(10):
    line1, line2, line3, line4, line5 = limirick_machine()
    temp_limirick_dict['line1'].append(line1)
    temp_limirick_dict['line2'].append(line2)
    temp_limirick_dict['line3'].append(line3)
    temp_limirick_dict['line4'].append(line4)
    temp_limirick_dict['line5'].append(line5)
basic_limirick_df = pd.DataFrame(temp_limirick_dict)


    
    

<class 'list'>
['G EY1 T']
['Beyond the temple gate']
EY1 T
dict_keys(['late', 'state', 'mate', 'fate', 'wait', 'great', 'hate', 'gate', 'waite', 'relate', 'debate', 'sate', 'create', 'date', 'grate', 'gait', 'weight', 'straight', 'await', 'innate', 'translate', 'plait', 'estate', 'eight', 'trait', 'strait', 'pate', 'ate', 'plate', 'kate', 'slate', 'dictate', 'freight', 'dilate', 'bait', 'rate', 'abate', 'elate', 'tate', 'bate', 'fete', 'reinstate', 'sedate', 'wayt', 'streight', 'inflate', 'spate', 'desecrate', 'ornate'])
<class 'list'>
['W EY1 JH']
['Beyond the temple gate', 'To grapple with the mysteries I late', 'Leave we the Trojans and the Greeks to wage']
EY1 JH
dict_keys(['rage', 'age', 'engage', 'page', 'disengage', 'stage', 'assuage', 'wage', 'cage', 'gauge', 'sage', 'gage'])
['Beyond the temple gate', 'To grapple with the mysteries I late', 'Leave we the Trojans and the Greeks to wage', 'Meanwhile the Soldan strove his rage']
EY1 T
dict_keys(['late', 'state', 'mate', 'fate', 

In [387]:
print(basic_limirick_df.shape)
print(basic_limirick_df.sample(1))

(10, 5)
                                       line1  \
1  Still, though your wickedness must harrow   

                                  line2                           line3  \
1  Called them forth with bow and arrow  Blubbering, to his aged master   

                                         line4  \
1  Is of yourself.  Tell me, who is the master   

                                      line5  
1  No point in Heaven's Azure but his Arrow  


In [419]:
test_value = basic_limirick_df['line1'].iloc[0]
test_value

'Beyond the temple gate'

In [421]:
type(TextBlob(test_value).sentiment)

textblob.en.sentiments.Sentiment

In [388]:
def sentimenter(line):
    return TextBlob(line).sentiment
for column in basic_limirick_df:
    basic_limirick_df[column + '_sentiment'], basic_limirick_df[column + '_polarity'] = basic_limirick_df[column].apply(sentimenter)
    print(column)

ValueError: too many values to unpack (expected 2)

In [ ]:
#make a wordcloud, preferably for the phones rhymegroup
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["br", "href"])
textt = " ".join(review for review in df.Text)
wordcloud = WordCloud(stopwords=stopwords).generate(textt)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud11.png')
plt.show()